In [8]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append("../")
from data_perturbations import *
from competing_methods import *
from PCS_confidence_intervals import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from imodels import FIGSRegressor
import scipy.stats as st
from sim_utils import *
import statistics
from tqdm import tqdm
#from imodels.importance import RandomForestPlusRegressor

In [2]:
def get_length(confidence_interval):
    return np.abs(confidence_interval[1] - confidence_interval[0])

# Load Data 

In [96]:
X  = pd.read_csv("../data/X_uncorrelated_enhancer.csv").iloc[:500,:]
num_reps = 200
heritability = 0.1
error_fun = np.random.normal
#y = pd.read_csv("../data/y_enhancer.csv")
#X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)
#X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25,)

# Evaluate Classical Confidence Interval 

In [97]:
classic_CI_length = []
classic_coverage_indicator = []
for i in range(num_reps):
    y = linear_model(np.array(X),sigma = None, s = 1, beta= 0.2, heritability = heritability,error_fun=error_fun)
    #print(classic_confidence_interval(X,y))
    CI = classic_confidence_interval(X,y).iloc[1,:].values
    classic_CI_length.append(get_length(CI))
    if CI[0] <= 0.2 <= CI[1]:
        classic_coverage_indicator.append(1.0)
    else:
        classic_coverage_indicator.append(0.0)


In [98]:
print(statistics.mean(classic_CI_length))
print(statistics.mean(classic_coverage_indicator))

0.15370272249997702
0.925


# Evaluate Bootstrap CI 

In [99]:
import warnings
warnings.filterwarnings("ignore")
bootstrap_CI_length = []
bootstrap_coverage_indicator = []
for i in range(num_reps):
    y = linear_model(np.array(X),sigma =  None, s = 1, beta= 0.2, heritability = heritability,error_fun=error_fun)
    bootstrap_confidence_intervals = bootstrap_confidence_interval(X,y)
    CI = bootstrap_confidence_intervals[0]
    bootstrap_CI_length.append(get_length(CI))
    if CI[0] <= 0.2 <= CI[1]:
        bootstrap_coverage_indicator.append(1.0)
    else:
        bootstrap_coverage_indicator.append(0.0)


In [100]:
print(statistics.mean(bootstrap_CI_length))
print(statistics.mean(bootstrap_coverage_indicator))

0.15440936984627077
0.94


# PCS Prediction Intervals

In [101]:
def _generate_perturbed_data(X,y):
    X_perturbed, y_perturbed = bootstrap(X,y)
    #X_perturbed = add_X_noise(X_perturbed)
    y_perturbed = add_normal_measurement_noise(y_perturbed)
    return X_perturbed,y_perturbed

In [102]:
def generate_perturbed_data(X,y):
    num_perturbations = 50
    X_train_perturbed_list = []
    y_train_perturbed_list = []
    for i in range(num_perturbations):
        X_perturbed,y_perturbed =  _generate_perturbed_data(X,y)
        X_train_perturbed_list.append(X_perturbed)
        y_train_perturbed_list.append(y_perturbed)
    return X_train_perturbed_list,y_train_perturbed_list

In [103]:
PCS_CI_length = []
PCS_coverage_indicator = []
for i in tqdm(range(num_reps)):
    y = linear_model(np.array(X),sigma = None, s = 1, beta= 0.2, heritability = heritability, error_fun=error_fun)
    X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)
    predictability_screening(X_train,y_train,X_test,y_test)
    X_train_perturbed_list,y_train_perturbed_list = generate_perturbed_data(X,y)
    import warnings
    warnings.filterwarnings("ignore")
    p_screened_coefficients = fit_all_model_perturbed_datasets(["OLS","RidgeCV"], X_train_perturbed_list, y_train_perturbed_list)
    CI = compute_confidence_intervals(X_train,p_screened_coefficients)[0,:]
    PCS_CI_length.append(get_length(CI))
    if CI[0] <= 0.2 <= CI[1]:
        PCS_coverage_indicator.append(1.0)
    else:
        PCS_coverage_indicator.append(0.0)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [03:22<00:00,  1.01s/it]


In [104]:
print(statistics.mean(PCS_CI_length))
print(statistics.mean(PCS_coverage_indicator))

0.14568875586043964
0.905
